## 1. Create the data frames for vertices, edges, and then create graph.

In [1]:
from graphframes import *

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [3]:
import os
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
spark = SparkSession.builder.appName("graph processing")\
.config("spark.jars.packages","graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
.master("local[*]").getOrCreate()

25/04/12 14:03:02 WARN Utils: Your hostname, bigdata2024 resolves to a loopback address: 127.0.1.1; using 10.3.132.222 instead (on interface ens3)
25/04/12 14:03:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hadoop/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4771b758-128e-48de-a0c0-9ba510b40d05;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 122ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

In [4]:
stations = spark.read.csv("station.csv", header=True, inferSchema=True)
trips = spark.read.csv("trip.csv", header=True, inferSchema=True)

stations.show(3)
trips.show(3)

+---+--------------------+---------+-------------------+----------+--------+-----------------+
| id|                name|      lat|               long|dock_count|    city|installation_date|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|37.333988|        -121.894902|        11|San Jose|         8/6/2013|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
only showing top 3 rows

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_co

In [5]:
vertices = stations.selectExpr("id as id", "name", "lat", "long", "dock_count", "city", "installation_date")
edges = trips.selectExpr("start_station_id as src", "end_station_id as dst", "duration")

vertices.show(3)
edges.show(3)

+---+--------------------+---------+-------------------+----------+--------+-----------------+
| id|                name|      lat|               long|dock_count|    city|installation_date|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|37.333988|        -121.894902|        11|San Jose|         8/6/2013|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
only showing top 3 rows

+---+---+--------+
|src|dst|duration|
+---+---+--------+
| 66| 66|      63|
| 10| 10|      70|
| 27| 27|      71|
+---+---+--------+
only showing top 3 rows



In [6]:
g = GraphFrame(vertices, edges)

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


## 2. Find the indegree and outdegree of all  stations.

In [7]:
in_degree = g.inDegrees
out_degree = g.outDegrees

print("In-Degree:")
in_degree.show()

print("Out-Degree:")
out_degree.show()

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In-Degree:


+---+--------+
| id|inDegree|
+---+--------+
| 31|    1832|
| 65|   26637|
| 34|    1819|
| 28|    6719|
| 76|   19915|
| 27|    3284|
| 26|     452|
| 12|    1437|
| 22|    1255|
| 47|    9269|
| 13|    1488|
|  6|    3163|
| 16|    1473|
|  3|    1786|
| 57|   14849|
| 54|   13144|
| 48|   12313|
|  5|    1169|
| 64|   16843|
| 41|    9936|
+---+--------+
only showing top 20 rows

Out-Degree:
+---+---------+
| id|outDegree|
+---+---------+
| 31|     1818|
| 65|    23724|
| 34|     2111|
| 28|     6544|
| 76|    20165|
| 27|     2984|
| 26|      311|
| 12|     1170|
| 22|     1554|
| 47|     9463|
| 13|     1680|
|  6|     2917|
| 16|     1473|
|  3|     1594|
| 57|    13526|
| 54|    14811|
| 48|    10592|
|  5|     1257|
| 64|    18496|
| 41|     9994|
+---+---------+
only showing top 20 rows



## 3. Find any two stations that have distance greater than 5 km.

In [8]:
from pyspark.sql.functions import col, radians, sin, cos, sqrt, asin

station_pairs = stations.alias("a").crossJoin(stations.alias("b")).filter("a.id < b.id")  

earth_radius = 6371  # km

distances = station_pairs.selectExpr(
    "a.id as id1", "b.id as id2",
    "a.name as name1", "b.name as name2",
    "a.lat as lat1", "a.long as lon1",
    "b.lat as lat2", "b.long as lon2"
).withColumn(
    "distance",
    earth_radius * 2 * asin(sqrt(
        sin((radians(col("lat2") - col("lat1")))/2)**2 +
        cos(radians(col("lat1"))) * cos(radians(col("lat2"))) *
        sin((radians(col("lon2") - col("lon1")))/2)**2
    ))
)

distances.filter("distance > 5").show(2)

+---+---+--------------------+--------------------+---------+-------------------+------------------+-------------------+-----------------+
|id1|id2|               name1|               name2|     lat1|               lon1|              lat2|               lon2|         distance|
+---+---+--------------------+--------------------+---------+-------------------+------------------+-------------------+-----------------+
|  2| 21|San Jose Diridon ...|   Franklin at Maple|37.329732|-121.90178200000001|         37.481758|        -122.226904|33.32341902182416|
|  2| 22|San Jose Diridon ...|Redwood City Calt...|37.329732|-121.90178200000001|37.486078000000006|-122.23208899999999|33.96153918542211|
+---+---+--------------------+--------------------+---------+-------------------+------------------+-------------------+-----------------+
only showing top 2 rows



## 4. Find any two station A,C that are connected by any one hop B and have the  total distance is greater than 150.

In [9]:
triplets = g.edges.alias("e1") \
    .join(g.edges.alias("e2"), col("e1.dst") == col("e2.src")) \
    .select(
        col("e1.src").alias("A"),
        col("e1.dst").alias("B"),
        col("e2.dst").alias("C"),
        (col("e1.duration") + col("e2.duration")).alias("total_duration")
    )

triplets.filter("total_duration > 150").show(5)

[Stage 17:==============>                                           (1 + 3) / 4]

+---+---+---+--------------+
|  A|  B|  C|total_duration|
+---+---+---+--------------+
|  5|  3|  8|           376|
|  5|  3|  6|           573|
|  5|  3|  2|           597|
|  5|  3| 13|           783|
|  5|  3| 13|           797|
+---+---+---+--------------+
only showing top 5 rows



## 5. Run PageRank to find the importance of all stations.

In [10]:
results = g.pageRank(resetProbability=0.15, maxIter=10)

results.vertices.select("id", "name", "pagerank").orderBy("pagerank", ascending=False).show(10)

+---+--------------------+------------------+
| id|                name|          pagerank|
+---+--------------------+------------------+
|  2|San Jose Diridon ...|3.5440610635004215|
| 70|San Francisco Cal...|3.2563572520336965|
| 28|Mountain View Cal...|2.3963182860250067|
| 22|Redwood City Calt...|    2.222023327527|
| 69|San Francisco Cal...|1.8817145911284543|
| 50|Harry Bridges Pla...|1.8146546040805573|
| 60|Embarcadero at Sa...|1.6784180823085506|
| 61|     2nd at Townsend|1.5408450673682548|
| 77|   Market at Sansome|1.4703197568612667|
| 65|     Townsend at 7th|1.4597489702478772|
+---+--------------------+------------------+
only showing top 10 rows

